In [1]:
import pandas as pd
from scipy import stats as st
import math
import random

In [2]:
df = pd.read_csv("AB_test_data.csv")

# inspect data frame
print(df.head())

  Variant  purchase_TF        date        id
0       A        False  2019-12-26  0x6f9421
1       A        False  2019-08-16  0x59d442
2       A         True  2019-03-18  0x6db8f8
3       A        False  2019-02-13  0x68245d
4       A        False  2019-09-28  0x28566e


## Question 1

In [3]:
# for variant A - what % purchased
p_pop = len(df[(df['Variant'] == "A") & (df['purchase_TF'] == True)]) / len(df[df['Variant'] == "A"])
print("Conversion_A: %.3f" % p_pop)

ConversionRate_A: 0.15


In [4]:
# for variant B - what % purchase
p_sam = len(df[(df['Variant'] == "B") & (df['purchase_TF'] == True)]) / len(df[df['Variant'] == "B"])
print("Conversion_B: %.3f" % p_sam)

Conversion_B: 0.196


In [5]:
# sample size - how many people got variant B
n = len(df[df['Variant'] == "B"])
print("n: %d" % n)

n: 5000


In [6]:
z = (p_sam - p_pop) / math.sqrt( (p_pop*(1-p_pop)) / n )
print("z-score: %.2f" % z)

z-score: 8.69


In [7]:
alpha = 0.05

z_alpha = st.norm.ppf(1-(alpha/2))

print("z_alpha2: %.2f" % z_alpha)

z_alpha2: 1.96


In [9]:
print("Reject Null Hypothesis: %s" % (z > z_alpha))

Reject Null Hypothesis: True


## Question 2

In [10]:
beta = 1-0.8
z_beta = st.norm.ppf(1-beta)
print("z_beta: %.2f" % z_beta)

z_beta: 0.84


In [15]:
delta = (p_sam - p_pop) #/ p_pop

print("Delta: %.3f" % delta)

Delta: 0.044


In [16]:
p_hat = (p_sam+p_pop)/2

print("p-hat: %.2f" % p_hat)

p-hat: 0.17


In [17]:
n_opt = ((z_alpha*math.sqrt(2*p_hat*(1-p_hat)) + z_beta * math.sqrt(p_pop*(1-p_pop) + p_sam *(1-p_sam)))**2 ) / delta**2

print("Optimal sample size: %d" % math.ceil(n_opt))

Optimal sample size: 1158


In [19]:
# create 10 samples according to size n_opt
samples = []

x=0
while x < 10:
    samples.append(random.sample(df[df['Variant'] == "B"]['id'].tolist(),math.ceil(n_opt)))
    x += 1

In [20]:
# test the samples
list_p_sample = []
list_z = []
list_reject_null = []

for i in samples:
    # subset data to just the samples
    df_sample = df[df['id'].isin(i)]
    
    # calculate p sample
    p_sample = len(df_sample[df_sample['purchase_TF'] == True]) / len(df_sample)
    
    # calculate z
    z = (p_sample - p_pop) / math.sqrt( (p_pop*(1-p_pop)) / math.ceil(n_opt) )
    
    # do i reject the null hypothesis
    reject_null = abs(z) > z_alpha
    
    list_p_sample.append(p_sample)
    list_z.append(z)
    list_reject_null.append(reject_null)
    
df_sampleresults = pd.DataFrame({'Sample_Number': range(0,len(samples)), 
                                 'p_sample': list_p_sample,
                                'z_score': list_z,
                                'reject_null': list_reject_null})

print(df_sampleresults)

   Sample_Number  p_sample   z_score  reject_null
0              0  0.189983  3.593882         True
1              1  0.181347  2.775501         True
2              2  0.188256  3.430206         True
3              3  0.199482  4.494101         True
4              4  0.203800  4.903292         True
5              5  0.196028  4.166749         True
6              6  0.193437  3.921234         True
7              7  0.197755  4.330425         True
8              8  0.198618  4.412263         True
9              9  0.186528  3.266529         True


## Question 3

In [21]:
# establish boundaries
upper_bound = math.log(1/alpha)
lower_bound = math.log(beta)

print("Upper Bound: %.2f" % upper_bound)
print("Lower Bound: %.2f" % lower_bound)

Upper Bound: 3.00
Lower Bound: -1.61


In [23]:
stopping_iteration = []
stopping_reason = []

for sample_n in samples:
    results = df[df['id'].isin(sample_n)]['purchase_TF'].values

    log_gamma = 0
    count = 0
    while (log_gamma > lower_bound) & (log_gamma < upper_bound):
        if results[count] == True:
            log_gamma = log_gamma + math.log(p_sam / p_pop)
        else:
            log_gamma = log_gamma + math.log( (1-p_sam) / (1-p_pop))

        count += 1

    stopping_iteration.append(count)
    
    if log_gamma < lower_bound:
        stopping_reason.append('Lower bound')
    else:
        stopping_reason.append('Upper bound')
    
print(stopping_iteration)
print(stopping_reason)
print("Average iterations: %.2f" % (sum(stopping_iteration) / len(stopping_iteration)))

[106, 65, 206, 445, 124, 190, 290, 302, 300, 1003]
['Lower bound', 'Upper bound', 'Upper bound', 'Upper bound', 'Upper bound', 'Upper bound', 'Upper bound', 'Lower bound', 'Upper bound', 'Upper bound']
Average iterations: 303.10
